In [16]:
!pip install tensorflow-addons

In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn import tree, ensemble, model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# NN
import tensorflow as tf
import tensorflow_hub as hub

from keras.callbacks import EarlyStopping


In [18]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [19]:
train = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/train.csv')
train

X_train = train.drop("change_lead", axis = 1)
y_train = train["change_lead"]

In [20]:
test = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/test.csv')
X_test = test.drop("change_lead", axis = 1)
y_test = test["change_lead"]

# Helper Function

In [21]:
agg_accuracy_score_dict = {}
agg_f1_score_dict={}
agg_roc_auc_dict={}

train_accuracy_score_dict = {}
train_f1_score_dict={}
train_roc_auc_dict={}

val_accuracy_score_dict = {}
val_f1_score_dict={}
val_roc_auc_dict={}

test_accuracy_score_dict = {}
test_f1_score_dict={}
test_roc_auc_dict={}

def evaluation(true, pred, title):
    print("================== Evaluation on {} ==================".format(title))
    # accuracy 
    acc = accuracy_score(true, pred)
    print("accuracy: {}\n".format(acc))
    agg_accuracy_score_dict[title] = acc
    # f1 score
    print("classification report: \n{}\n".format(classification_report(true, pred)))
    print("F1 score: \n{}\n".format(metrics.f1_score(true, pred)))
    f1 = f1_score(true, pred)
    agg_f1_score_dict[title] = f1
    # confusion matrix
    print("Confusion matrix: \n{}\n".format(confusion_matrix(true, pred)))
    # roc auc result
    fpr, tpr, thresholds = roc_curve(true, pred)
    roc_auc = auc(fpr, tpr)
    print("ROC AUC: {}\n".format(roc_auc))
    agg_roc_auc_dict[title] = roc_auc
    print("================== End of Evaluation on {} ==================".format(title))
    
    if "train dataset" in title:
        train_accuracy_score_dict[title] = acc
        train_f1_score_dict[title] = f1
        train_roc_auc_dict[title] = roc_auc
    elif "test dataset" in title:
        test_accuracy_score_dict[title] = acc
        test_f1_score_dict[title] = f1
        test_roc_auc_dict[title] = roc_auc
    else:
        val_accuracy_score_dict[title] = acc
        val_f1_score_dict[title] = f1
        val_roc_auc_dict[title] = roc_auc

# NN for all companies

## Baseline

In [7]:
# baseline model
def create_baseline():
	# create model
	model = tf.keras.Sequential()
	model.add(tf.keras.layers.Dense(60, input_shape=(13,), activation='relu'))
	model.add(tf.keras.layers.Dense(24, activation='relu'))
	model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='SGD',metrics=['accuracy'])
	return model

baseline_tf = create_baseline()

# BASELINE MODEL, NO EARLY STOPPING
# baseline_tf.fit(X_train, y_train, epochs=100, batch_size=50,  verbose=1, validation_split=0.1,)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5, restore_best_weights = True)
baseline_tf.fit(X_train, y_train, batch_size = 50, 
                    epochs = 100, validation_split = 0.1,
                    callbacks = [es])
baseline_ypred = baseline_tf.predict(X_test)

Epoch 1/100
45/45 [==============================] - 1s 6ms/step - loss: 0.6946 - accuracy: 0.5009 - val_loss: 0.6963 - val_accuracy: 0.4578
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6935 - accuracy: 0.5206 - val_loss: 0.6963 - val_accuracy: 0.4659
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6929 - accuracy: 0.5220 - val_loss: 0.6965 - val_accuracy: 0.4859
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 0.5206 - val_loss: 0.6965 - val_accuracy: 0.4659
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 0.6922 - accuracy: 0.5175 - val_loss: 0.6966 - val_accuracy: 0.4819
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 0.6920 - accuracy: 0.5215 - val_loss: 0.6967 - val_accuracy: 0.4819
Epoch 6: early stopping


In [8]:
yprednn = baseline_ypred.copy()
yprednn = pd.DataFrame(yprednn, columns=['y_pred_nn'])
yprednn['y_pred_nn'] = yprednn['y_pred_nn'].apply(lambda x: 0 if x < 0.5 else 1)
print(f1_score(y_test, yprednn['y_pred_nn']))
print(evaluation(y_test, yprednn['y_pred_nn'], "NN evaluation"))

0.5185185185185185
================== Evaluation on NN evaluation ==================
accuracy: 0.5289855072463768

classification report: 
              precision    recall  f1-score   support

           0       0.56      0.52      0.54       147
           1       0.50      0.54      0.52       129

    accuracy                           0.53       276
   macro avg       0.53      0.53      0.53       276
weighted avg       0.53      0.53      0.53       276


F1 score: 
0.5185185185185185

Confusion matrix: 
[[76 71]
 [59 70]]

ROC AUC: 0.5298212308179086

================== End of Evaluation on NN evaluation ==================
None


## Hyperparameter tuning

In [9]:
import tensorflow as tf
import tensorflow_hub as hub

from keras.callbacks import EarlyStopping
import tensorflow_addons as tfa

# baseline model
def create_baseline(hiddenLayerOne=60, hiddenLayerTwo=24,
	dropout=0.2, learnRate=0.01):
	# create model
	model = tf.keras.Sequential()
	model.add(tf.keras.layers.Dense(hiddenLayerOne, input_shape=(13,), activation='relu'))
	model.add(tf.keras.layers.Dense(hiddenLayerTwo, activation='relu'))
	model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=learnRate),metrics=['mse'])
	return model

In [10]:
# define a grid of the hyperparameter search space
hiddenLayerOne = [60, 32, 24]
hiddenLayerTwo = [32, 24, 12]
learnRate = [1e-2, 1e-3, 1e-4]
dropout = [0.3, 0.4, 0.5]
batchSize = [4, 8, 16, 32]
epochs = [10, 20, 30, 40]

# create a dictionary from the hyperparameter grid
grid = dict(
	hiddenLayerOne=hiddenLayerOne,
	learnRate=learnRate,
	hiddenLayerTwo=hiddenLayerTwo,
	# dropout=dropout,
	# batch_size=batchSize,
	# epochs=epochs
)

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# initialize a random search with a 3-fold cross-validation and then
# start the hyperparameter search process
print("[INFO] performing random search...")

model = KerasClassifier(build_fn=create_baseline, epochs=6, batch_size=20, verbose=1)

searcher = RandomizedSearchCV(estimator=model, n_jobs=-1, cv=3,
	param_distributions=grid, scoring="f1")
searchResults = searcher.fit(X_train, y_train)

# summarize grid search information
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
# print("[INFO] best score is {:.2f} using {}".format(bestScore,
# 	bestParams))

print("bestParams is",
	bestParams)

[INFO] performing random search...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Epoch 1/6


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


125/125 [==============================] - 1s 2ms/step - loss: 0.6952 - mse: 0.2510
Epoch 2/6
125/125 [==============================] - 0s 2ms/step - loss: 0.6936 - mse: 0.2502
Epoch 3/6
125/125 [==============================] - 0s 2ms/step - loss: 0.6928 - mse: 0.2498
Epoch 4/6
125/125 [==============================] - 0s 2ms/step - loss: 0.6923 - mse: 0.2496
Epoch 5/6
125/125 [==============================] - 0s 2ms/step - loss: 0.6919 - mse: 0.2494
Epoch 6/6
125/125 [==============================] - 0s 2ms/step - loss: 0.6917 - mse: 0.2493
bestParams is {'learnRate': 0.01, 'hiddenLayerTwo': 24, 'hiddenLayerOne': 60}


In [12]:
print("[INFO] evaluating the best model...")
bestModel = searchResults.best_estimator_
ypredz = bestModel.predict(X_test)
print(f1_score(y_test, ypredz))

[INFO] evaluating the best model...
0.6331658291457287


In [14]:
print(evaluation(y_test, ypredz, "NN evaluation"))

================== Evaluation on NN evaluation ==================
accuracy: 0.47101449275362317

classification report: 
              precision    recall  f1-score   support

           0       0.57      0.03      0.05       147
           1       0.47      0.98      0.63       129

    accuracy                           0.47       276
   macro avg       0.52      0.50      0.34       276
weighted avg       0.52      0.47      0.32       276


F1 score: 
0.6331658291457287

Confusion matrix: 
[[  4 143]
 [  3 126]]

ROC AUC: 0.5019775352001266

================== End of Evaluation on NN evaluation ==================
None


In [15]:
print(precision_score(y_test, ypredz))

0.4684014869888476


# NN for AAPL

## Baseline model

In [22]:
train_wx = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/train_wx.csv')
train_wx = train_wx[train_wx['symbol']=="AAPL"]
train_wx = train_wx.drop(columns=['symbol'])

X_train_wx = train_wx.drop("change_lead", axis = 1)
y_train_wx = train_wx["change_lead"]

In [23]:
test_wx = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/test_wx.csv')
test_wx = test_wx[test_wx['symbol']=='AAPL']
test_wx = test_wx.drop(columns=['symbol'])

X_test_wx = test_wx.drop("change_lead", axis = 1)
y_test_wx = test_wx["change_lead"]

In [24]:
# baseline model
def create_baseline():
	# create model
	model = tf.keras.Sequential()
	model.add(tf.keras.layers.Dense(60, input_shape=(13,), activation='relu'))
	model.add(tf.keras.layers.Dense(24, activation='relu'))
	model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='SGD',metrics=['accuracy'])
	return model

baseline_tf = create_baseline()

# BASELINE MODEL, NO EARLY STOPPING
# baseline_tf.fit(X_train_wx, y_train_wx, epochs=100, batch_size=50,  verbose=1, validation_split=0.1,)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5, restore_best_weights = True)
baseline_tf.fit(X_train_wx, y_train_wx, batch_size = 50, 
                    epochs = 100, validation_split = 0.1,
                    callbacks = [es])
baseline_ypred = baseline_tf.predict(X_test_wx)

Epoch 1/100
11/11 [==============================] - 1s 40ms/step - loss: 0.6929 - accuracy: 0.5342 - val_loss: 0.6908 - val_accuracy: 0.4590
Epoch 2/100
11/11 [==============================] - 0s 6ms/step - loss: 0.6925 - accuracy: 0.5471 - val_loss: 0.6919 - val_accuracy: 0.4590
Epoch 3/100
11/11 [==============================] - 0s 9ms/step - loss: 0.6921 - accuracy: 0.5416 - val_loss: 0.6928 - val_accuracy: 0.4590
Epoch 4/100
11/11 [==============================] - 0s 8ms/step - loss: 0.6918 - accuracy: 0.5434 - val_loss: 0.6938 - val_accuracy: 0.4426
Epoch 5/100
11/11 [==============================] - 0s 7ms/step - loss: 0.6915 - accuracy: 0.5416 - val_loss: 0.6947 - val_accuracy: 0.4426
Epoch 6/100
11/11 [==============================] - 0s 7ms/step - loss: 0.6913 - accuracy: 0.5471 - val_loss: 0.6955 - val_accuracy: 0.4426
Epoch 6: early stopping


In [25]:
yprednn = baseline_ypred.copy()
yprednn = pd.DataFrame(yprednn, columns=['y_pred_nn'])
yprednn['y_pred_nn'] = yprednn['y_pred_nn'].apply(lambda x: 0 if x < 0.5 else 1)
print(f1_score(y_test_wx, yprednn['y_pred_nn']))

0.725


In [26]:
evaluation(y_test_wx, yprednn['y_pred_nn'], "NN FOR AAPL")

================== Evaluation on NN FOR AAPL ==================
accuracy: 0.5686274509803921

classification report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.57      1.00      0.72        29

    accuracy                           0.57        51
   macro avg       0.28      0.50      0.36        51
weighted avg       0.32      0.57      0.41        51


F1 score: 
0.725

Confusion matrix: 
[[ 0 22]
 [ 0 29]]

ROC AUC: 0.5

================== End of Evaluation on NN FOR AAPL ==================


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Hyperparameter tuning

In [27]:
import tensorflow as tf
import tensorflow_hub as hub

from keras.callbacks import EarlyStopping
import tensorflow_addons as tfa

# baseline model
def create_baseline(hiddenLayerOne=60, hiddenLayerTwo=24,
	dropout=0.2, learnRate=0.01):
	# create model
	model = tf.keras.Sequential()
	model.add(tf.keras.layers.Dense(hiddenLayerOne, input_shape=(13,), activation='relu'))
	model.add(tf.keras.layers.Dense(hiddenLayerTwo, activation='relu'))
	model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=learnRate),metrics=['mse'])
	return model

In [28]:
# define a grid of the hyperparameter search space
hiddenLayerOne = [60, 32, 24]
hiddenLayerTwo = [32, 24, 12]
learnRate = [1e-2, 1e-3, 1e-4]
dropout = [0.3, 0.4, 0.5]
batchSize = [4, 8, 16, 32]
epochs = [10, 20, 30, 40]

# create a dictionary from the hyperparameter grid
grid = dict(
	hiddenLayerOne=hiddenLayerOne,
	learnRate=learnRate,
	hiddenLayerTwo=hiddenLayerTwo,
	# dropout=dropout,
	# batch_size=batchSize,
	# epochs=epochs
)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# initialize a random search with a 3-fold cross-validation and then
# start the hyperparameter search process
print("[INFO] performing random search...")

model = KerasClassifier(build_fn=create_baseline, epochs=6, batch_size=20, verbose=1)

searcher = RandomizedSearchCV(estimator=model, n_jobs=-1, cv=3,
	param_distributions=grid, scoring="f1")
searchResults = searcher.fit(X_train_wx, y_train_wx)

# summarize grid search information
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
# print("[INFO] best score is {:.2f} using {}".format(bestScore,
# 	bestParams))

print("bestParams is",
	bestParams)

[INFO] performing random search...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Epoch 1/6


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


31/31 [==============================] - 0s 2ms/step - loss: 0.7074 - mse: 0.2571
Epoch 2/6
31/31 [==============================] - 0s 2ms/step - loss: 0.7064 - mse: 0.2566
Epoch 3/6
31/31 [==============================] - 0s 1ms/step - loss: 0.7053 - mse: 0.2560
Epoch 4/6
31/31 [==============================] - 0s 1ms/step - loss: 0.7044 - mse: 0.2556
Epoch 5/6
31/31 [==============================] - 0s 2ms/step - loss: 0.7036 - mse: 0.2552
Epoch 6/6
31/31 [==============================] - 0s 1ms/step - loss: 0.7026 - mse: 0.2547
bestParams is {'learnRate': 0.001, 'hiddenLayerTwo': 32, 'hiddenLayerOne': 60}


In [ ]:
print("[INFO] evaluating the best model...")
bestModel = searchResults.best_estimator_
ypredz = bestModel.predict(X_test_wx)
print(f1_score(y_test_wx, ypredz))

[INFO] evaluating the best model...
0.0


# NN FOR MSFT

## Baseline model

In [ ]:
train_wx = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/train_wx.csv')
train_wx = train_wx[train_wx['symbol']=="MSFT"]
train_wx = train_wx.drop(columns=['symbol'])

X_train_wx = train_wx.drop("change_lead", axis = 1)
y_train_wx = train_wx["change_lead"]

In [ ]:
test_wx = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/test_wx.csv')
test_wx = test_wx[test_wx['symbol']=='MSFT']
test_wx = test_wx.drop(columns=['symbol'])

X_test_wx = test_wx.drop("change_lead", axis = 1)
y_test_wx = test_wx["change_lead"]

In [ ]:
# baseline model
def create_baseline():
	# create model
	model = tf.keras.Sequential()
	model.add(tf.keras.layers.Dense(60, input_shape=(13,), activation='relu'))
	model.add(tf.keras.layers.Dense(24, activation='relu'))
	model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='SGD',metrics=['accuracy'])
	return model

baseline_tf = create_baseline()

# BASELINE MODEL, NO EARLY STOPPING
# baseline_tf.fit(X_train_wx, y_train_wx, epochs=100, batch_size=50,  verbose=1, validation_split=0.1,)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5, restore_best_weights = True)
baseline_tf.fit(X_train_wx, y_train_wx, batch_size = 50, 
                    epochs = 100, validation_split = 0.1,
                    callbacks = [es])
baseline_ypred = baseline_tf.predict(X_test_wx)

Epoch 1/100
15/15 [==============================] - 1s 15ms/step - loss: 0.6963 - accuracy: 0.4980 - val_loss: 0.6943 - val_accuracy: 0.4756
Epoch 2/100
15/15 [==============================] - 0s 3ms/step - loss: 0.6957 - accuracy: 0.5156 - val_loss: 0.6938 - val_accuracy: 0.5244
Epoch 3/100
15/15 [==============================] - 0s 3ms/step - loss: 0.6950 - accuracy: 0.5279 - val_loss: 0.6935 - val_accuracy: 0.5244
Epoch 4/100
15/15 [==============================] - 0s 3ms/step - loss: 0.6946 - accuracy: 0.5265 - val_loss: 0.6933 - val_accuracy: 0.5244
Epoch 5/100
15/15 [==============================] - 0s 3ms/step - loss: 0.6941 - accuracy: 0.5279 - val_loss: 0.6931 - val_accuracy: 0.5244
Epoch 6/100
15/15 [==============================] - 0s 3ms/step - loss: 0.6935 - accuracy: 0.5265 - val_loss: 0.6929 - val_accuracy: 0.5366
Epoch 7/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5320 - val_loss: 0.6928 - val_accuracy: 0.5366
Epoch 8/100


In [ ]:
yprednn = baseline_ypred.copy()
yprednn = pd.DataFrame(yprednn, columns=['y_pred_nn'])
yprednn['y_pred_nn'] = yprednn['y_pred_nn'].apply(lambda x: 0 if x < 0.5 else 1)
print(f1_score(y_test_wx, yprednn['y_pred_nn']))

0.6236559139784946


In [ ]:
evaluation(y_test_wx, yprednn['y_pred_nn'], "NN FOR GOOG")

================== Evaluation on NN FOR GOOG ==================
accuracy: 0.453125

classification report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        35
           1       0.45      1.00      0.62        29

    accuracy                           0.45        64
   macro avg       0.23      0.50      0.31        64
weighted avg       0.21      0.45      0.28        64


F1 score: 
0.6236559139784946

Confusion matrix: 
[[ 0 35]
 [ 0 29]]

ROC AUC: 0.5

================== End of Evaluation on NN FOR GOOG ==================


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Hyperparameter tuning

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

from keras.callbacks import EarlyStopping
import tensorflow_addons as tfa

# baseline model
def create_baseline(hiddenLayerOne=60, hiddenLayerTwo=24,
	dropout=0.2, learnRate=0.01):
	# create model
	model = tf.keras.Sequential()
	model.add(tf.keras.layers.Dense(hiddenLayerOne, input_shape=(13,), activation='relu'))
	model.add(tf.keras.layers.Dropout(rate=0.25))
	model.add(tf.keras.layers.Dense(hiddenLayerTwo, activation='relu'))
	model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=learnRate),metrics=['mse'])
	return model

In [ ]:
# define a grid of the hyperparameter search space
hiddenLayerOne = [120, 72, 52]
hiddenLayerTwo = [32, 24, 12]
learnRate = [1e-2, 1e-3, 1e-4]
dropout = [0.3, 0.4, 0.5]
batchSize = [4, 8, 16, 32]
epochs = [10, 20, 30, 40]

# create a dictionary from the hyperparameter grid
grid = dict(
	hiddenLayerOne=hiddenLayerOne,
	learnRate=learnRate,
	hiddenLayerTwo=hiddenLayerTwo,
	# dropout=dropout,
	# batch_size=batchSize,
	# epochs=epochs
)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# initialize a random search with a 3-fold cross-validation and then
# start the hyperparameter search process
print("[INFO] performing random search...")

model = KerasClassifier(build_fn=create_baseline, epochs=6, batch_size=20, verbose=1)

searcher = RandomizedSearchCV(estimator=model, n_jobs=-1, cv=3,
	param_distributions=grid, scoring="f1")
searchResults = searcher.fit(X_train_wx, y_train_wx)

# summarize grid search information
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
# print("[INFO] best score is {:.2f} using {}".format(bestScore,
# 	bestParams))

print("bestParams is",
	bestParams)

[INFO] performing random search...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Epoch 1/6


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


41/41 [==============================] - 0s 2ms/step - loss: 0.6961 - mse: 0.2515
Epoch 2/6
41/41 [==============================] - 0s 2ms/step - loss: 0.6946 - mse: 0.2507
Epoch 3/6
41/41 [==============================] - 0s 2ms/step - loss: 0.6931 - mse: 0.2500
Epoch 4/6
41/41 [==============================] - 0s 2ms/step - loss: 0.6926 - mse: 0.2497
Epoch 5/6
41/41 [==============================] - 0s 2ms/step - loss: 0.6928 - mse: 0.2498
Epoch 6/6
41/41 [==============================] - 0s 2ms/step - loss: 0.6907 - mse: 0.2488
bestParams is {'learnRate': 0.01, 'hiddenLayerTwo': 24, 'hiddenLayerOne': 72}


In [ ]:
print("[INFO] evaluating the best model...")
bestModel = searchResults.best_estimator_
ypredz = bestModel.predict(X_test_wx)
print(f1_score(y_test_wx, ypredz))

[INFO] evaluating the best model...
0.6236559139784946
